In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

Load anndata

In [2]:
adata = sc.read_h5ad("test.h5ad")

In [3]:
# lognorm should be the default (expected)
adata.X = adata.layers['logcounts']

Parameters

In [4]:
groupby='label'
de_method='wilcoxon'
resource_name = 'consensus'
complex_policy = 'min'
key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']
verbose = False

In [5]:
# only relevant if perms are done
n_perms = 1000
seed = 69

Run LIANA PIPE

In [6]:
from liana import liana_pipe

In [7]:
lr_res = liana_pipe(adata=adata, 
                    groupby=groupby, 
                    resource_name=resource_name,
                    de_method=de_method,
                    n_perms=n_perms,
                    seed=seed,
                    verbose=verbose,
                    _key_cols=key_cols,
                    resource=None)

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


In [8]:
lr_res.sort_values('ligand_means')

,ligand,ligand_complex,ligand_logfc,ligand_means,ligand_zscores,mat_mean,receptor,receptor_complex,receptor_logfc,receptor_means,receptor_zscores,source,target,ligand_means_sums,receptor_means_sums
2288,CSF1,CSF1,-0.044315,0.000000,-0.105409,0.199056,CSF2RA,CSF2RA,-0.055999,0.000000,-0.105409,CD8 T,CD8 T,0.035162,0.040544
2096,BTLA,BTLA,-0.048226,0.000000,-0.105409,0.199056,TNFRSF14,TNFRSF14,-0.520295,0.294117,-0.250196,CD8 T,CD8 T,0.037057,1.424242
2087,IL18BP,IL18BP,-0.132456,0.000000,-0.145695,0.199056,IL6R,IL6R,-0.068199,0.000000,-0.105409,CD8 T,CD8 T,0.087242,0.045404
2086,IL6,IL6,-0.315346,0.000000,-0.149471,0.199056,IL6ST,IL6R_IL6ST,-0.121447,0.000000,-0.146079,CD8 T,CD8 T,0.140459,0.083360
2085,IL6,IL6,-0.315346,0.000000,-0.149471,0.199056,IL6R,IL6R_IL6ST,-0.068199,0.000000,-0.105409,CD8 T,CD8 T,0.140459,0.045404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,B2M,B2M,0.595277,5.810702,0.728883,0.199056,CD247,CD247,-2.154735,0.269538,-0.764050,NK,B,16.490938,3.277351
4178,B2M,B2M,0.595277,5.810702,0.728883,0.199056,CD3G,CD3G,-0.160309,0.196046,-0.106673,NK,NK,16.490938,0.797412
4332,B2M,B2M,0.595277,5.810702,0.728883,0.199056,KLRC1,KLRC1,1.285116,0.642471,0.494169,NK,NK,16.490938,0.893647
2647,B2M,B2M,0.595277,5.810702,0.728883,0.199056,CD3D,CD3D,2.097606,2.190063,0.951525,NK,CD8 T,16.490938,3.070996


Consensus

In [9]:
from liana.steady import cellphonedb

In [10]:
from liana.steady.scores.consensus import consensus

In [11]:
consensus.complex_cols

['receptor_means',
 'receptor_zscores',
 'ligand_means',
 'ligand_logfc',
 'ligand_zscores',
 'receptor_logfc']

In [12]:
adata = consensus(adata, groupby='label')

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/anndata/compat/_overloaded_dict.py:106: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  self.data[key] = value
/home/dbdimitrov/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


CellPhoneDB
Index(['ligand', 'ligand_complex', 'ligand_means', 'receptor',
       'receptor_complex', 'receptor_means', 'source', 'target'],
      dtype='object')
Connectome
Index(['ligand', 'ligand_complex', 'ligand_means', 'ligand_zscores',
       'receptor', 'receptor_complex', 'receptor_means', 'receptor_zscores',
       'source', 'target'],
      dtype='object')
log2FC
Index(['ligand', 'ligand_complex', 'ligand_logfc', 'ligand_means', 'receptor',
       'receptor_complex', 'receptor_logfc', 'receptor_means', 'source',
       'target'],
      dtype='object')
NATMI
Index(['ligand', 'ligand_complex', 'ligand_means', 'ligand_means_sums',
       'receptor', 'receptor_complex', 'receptor_means', 'receptor_means_sums',
       'source', 'target'],
      dtype='object')
SingleCellSignalR
Index(['ligand', 'ligand_complex', 'ligand_means', 'mat_mean', 'receptor',
       'receptor_complex', 'receptor_means', 'source', 'target'],
      dtype='object')


In [13]:
adata.uns['liana_res']['CellPhoneDB']

,ligand,ligand_complex,ligand_means,receptor,receptor_complex,receptor_means,source,target,lr_means,pvals
0,LGALS9,LGALS9,0.374320,PTPRC,PTPRC,0.761533,B,B,0.567927,0.859
1,CD22,CD22,0.554265,PTPRC,PTPRC,0.761533,B,B,0.657899,0.595
2,LGALS1,LGALS1,0.611330,PTPRC,PTPRC,0.761533,B,B,0.686431,1.000
3,LGALS9,LGALS9,0.374320,CD44,CD44,0.655391,B,B,0.514855,0.141
4,ADAM10,ADAM10,0.119797,CD44,CD44,0.655391,B,B,0.387594,0.598
...,...,...,...,...,...,...,...,...,...,...
4576,CALM1,CALM1,2.516959,KCNN4,KCNN4,0.000000,NK,NK,1.000000,1.000
4577,PTPN6,PTPN6,0.652221,CD300LF,CD300LF,0.000000,NK,NK,1.000000,1.000
4578,NUCB2,NUCB2,0.156951,ERAP1,ERAP1,0.162055,NK,NK,0.159503,0.069
4579,SOCS2,SOCS2,0.230338,EPOR,EPOR,0.000000,NK,NK,1.000000,1.000


In [14]:
from liana.steady import cellphonedb, natmi, connectome, logfc, singlecellsignalr as sca

In [15]:
_methods = [cellphonedb, connectome, logfc, natmi, sca]

Robust rank aggregate

In [ ]:
# Duplicates arrise due to the explode of complexes, in which subunits have the same min expression
# these are kept when returning the result from each method but are discarded 
# when working with the consensus because duplicated interactions are assigned the same score

In [16]:
from functools import reduce

In [17]:
from scipy.stats import rankdata

In [18]:
from scipy.stats import beta

In [19]:
def corr_beta_pvals(p, k):
    p = np.clip(p * k, a_min=0, a_max=1)
    return p

In [20]:
lr_res = adata.uns['liana_res'].copy()

In [21]:
keys = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [ ]:
# 1. Get all keys

In [22]:
# get all shared keys
shared_keys = [lr_res[method.method_name][keys].drop_duplicates(keep='first') for method in _methods]
# outer join all from different methods to keep all possibilities
shared_keys = reduce(lambda left, right: pd.merge(left, right, how='outer', on=keys), shared_keys)

In [23]:
# 2. Set-up the scores to be used (method_name:(score_name, desc))
specs = {method.method_name:(method.specificity, method.specificity_desc) for method in _methods if method.specificity is not None}
# mags = {method.method_name:method.magnitude for method in _methods if method.magnitude is not None}
# steady = specs + {sca.method_name:sca.magnitude}

In [24]:
# steady
specs[sca.method_name]=(sca.magnitude, sca.magnitude_desc)

In [25]:
specs

{'CellPhoneDB': ('pvals', False),
 'Connectome': ('scaled_weight', True),
 'log2FC': ('lr_logfc', True),
 'NATMI': ('spec_weight', True),
 'SingleCellSignalR': ('lrscore', True)}

In [ ]:
# 3. To them join the scores

In [26]:
res = [ lr_res[spec][keys + [specs[spec][0]]].drop_duplicates(keep='first') for spec in specs ]

In [27]:
# reduce to a df with the shared keys + all relevant scores
res = reduce(lambda left, right: pd.merge(left, right, how='left', on=keys), [shared_keys] + res)

In [28]:
res

,source,target,ligand_complex,receptor_complex,pvals,scaled_weight,lr_logfc,spec_weight,lrscore
0,B,B,LGALS9,PTPRC,0.859,-0.018851,-0.128937,0.156310,0.728423
1,B,B,CD22,PTPRC,0.595,0.101330,-0.031155,0.184109,0.765469
2,B,B,LGALS1,PTPRC,1.000,-0.438067,-1.391997,0.038055,0.774150
3,B,B,LGALS9,CD44,0.141,0.163107,0.267569,0.218076,0.713324
4,B,B,ADAM10,CD44,0.598,-0.027165,-0.127467,0.102962,0.584658
...,...,...,...,...,...,...,...,...,...
4270,NK,NK,CALM1,KCNN4,1.000,0.171759,0.344996,0.000000,0.000000
4271,NK,NK,PTPN6,CD300LF,1.000,-0.152207,-0.361070,0.000000,0.000000
4272,NK,NK,NUCB2,ERAP1,0.069,0.174068,0.281153,0.313999,0.444812
4273,NK,NK,SOCS2,EPOR,1.000,0.068864,0.220448,0.000000,0.000000


In [ ]:
# 4. Calculate Ranks

In [29]:
n = res.shape[0]

In [30]:
# Convert specs columns to ranks
for spec in specs:
    score_name = specs[spec][0]
    desc = specs[spec][1]
    
    if desc:
        res[score_name] = n - rankdata(res[score_name], method='min') # change to average
    else: 
        res[score_name] = rankdata(res[score_name], method='min') # change to average


Mean rank

In [31]:
rmat = res.drop(keys, axis=1).values

In [32]:
shared_keys['mean_rank'] = np.mean(rmat, axis=1)

In [33]:
shared_keys.sort_values('mean_rank')

,source,target,ligand_complex,receptor_complex,mean_rank
1584,B,CD8 T,HLA-DRA,LAG3,20.4
1586,B,CD8 T,HLA-DQA1,LAG3,27.0
1587,B,CD8 T,HLA-DRB1,LAG3,31.0
1590,B,CD8 T,HLA-DPA1,LAG3,31.8
1589,B,CD8 T,HLA-DPB1,LAG3,33.0
...,...,...,...,...,...
3257,B,NK,CCL5,CCRL2,3836.0
3248,B,NK,CCL5,CCR5,3837.6
1829,B,CD8 T,GZMA,F2R,3839.0
408,B,B,CCL5,DPP4,3839.2


Re-implement robust rank

In [34]:
rmat = res.drop(keys, axis=1).values / n ## DIVIDE BY THE MAX OF EACH NOT BY THE LENGTH (to deal /w min_rank stuff)

In [35]:
rmat

array([[0.41099415, 0.50666667, 0.64374269, 0.26573099, 0.11321637],
       [0.34245614, 0.26409357, 0.49356725, 0.22362573, 0.08421053],
       [0.49777778, 0.96818713, 0.98245614, 0.48912281, 0.07929825],
       ...,
       [0.14760234, 0.1677193 , 0.16350877, 0.10526316, 0.33333333],
       [0.49777778, 0.32210526, 0.19251462, 0.99976608, 0.99976608],
       [0.49777778, 0.52748538, 0.30526316, 0.99976608, 0.99976608]])

In [36]:
dist_a = np.repeat([np.arange(rmat.shape[1])], rmat.shape[0], axis=0) + 1

In [37]:
dist_b = rmat.shape[1] - dist_a + 1

In [47]:
# for 1 row (interaction)
corr_beta_pvals(np.min(beta.cdf(np.sort(rmat[2474]), dist_a[1], dist_b[1]), axis=0), k = rmat.shape[1])

0.0

In [39]:
p = beta.cdf(np.sort(rmat, axis=1), dist_a, dist_b)

In [40]:
p

array([[0.45161152, 0.40034148, 0.33660606, 0.19594443, 0.11055071],
       [0.35586301, 0.31169676, 0.11893479, 0.04992838, 0.02929083],
       [0.33840098, 0.7986078 , 0.49583339, 0.99050811, 0.91530505],
       ...,
       [0.42657503, 0.16038939, 0.03369399, 0.00342556, 0.00411523],
       [0.65670037, 0.5167354 , 0.49583339, 0.99999945, 0.99883096],
       [0.83815412, 0.80970988, 0.55143136, 0.99999945, 0.99883096]])

In [41]:
rho = corr_beta_pvals(np.min(p, axis=1), k = rmat.shape[1])

In [42]:
rho

array([0.55275357, 0.14645415, 1.        , ..., 0.01712782, 1.        ,
       1.        ])

In [43]:
shared_keys['aggregate_rank'] = rho

In [44]:
shared_keys[(shared_keys['ligand_complex']=='COL18A1') & (shared_keys['receptor_complex']=='ITGA5') ].sort_values('mean_rank')

,source,target,ligand_complex,receptor_complex,mean_rank,aggregate_rank
3107,B,NK,COL18A1,ITGA5,648.4,0.012891
1682,B,CD8 T,COL18A1,ITGA5,1269.0,0.135680
3582,CD8 T,NK,COL18A1,ITGA5,2638.2,1.000000
4057,NK,NK,COL18A1,ITGA5,2638.2,1.000000
257,B,B,COL18A1,ITGA5,2936.8,1.000000
2157,CD8 T,CD8 T,COL18A1,ITGA5,3226.4,1.000000
2632,NK,CD8 T,COL18A1,ITGA5,3226.4,1.000000
732,CD8 T,B,COL18A1,ITGA5,3476.2,1.000000
1207,NK,B,COL18A1,ITGA5,3476.2,1.000000


In [45]:
shared_keys[shared_keys['aggregate_rank']==0]

,source,target,ligand_complex,receptor_complex,mean_rank,aggregate_rank
1584,B,CD8 T,HLA-DRA,LAG3,20.4,0.0
2474,NK,CD8 T,B2M,CD3D,145.2,0.0


In [46]:
shared_keys.sort_values('aggregate_rank')

,source,target,ligand_complex,receptor_complex,mean_rank,aggregate_rank
2474,NK,CD8 T,B2M,CD3D,145.2,0.000000e+00
1584,B,CD8 T,HLA-DRA,LAG3,20.4,0.000000e+00
1586,B,CD8 T,HLA-DQA1,LAG3,27.0,6.397485e-10
1587,B,CD8 T,HLA-DRB1,LAG3,31.0,7.770483e-09
1590,B,CD8 T,HLA-DPA1,LAG3,31.8,1.298251e-08
...,...,...,...,...,...,...
1761,B,CD8 T,TGFB1,ACVR1_TGFBR2,3514.6,1.000000e+00
1765,B,CD8 T,TGFB1,TGFBR3,2347.4,1.000000e+00
1766,B,CD8 T,TGFB1,ENG,3462.4,1.000000e+00
1818,B,CD8 T,CCL5,CCR1,3832.0,1.000000e+00


In [68]:
# test rho_scores
a = np.array([0.1, 0.3, 0.5, 0.1, 0.3])
x = beta.cdf(np.sort(a), dist_a[1], dist_b[1])
corr_beta_pvals(np.min(x), 5)
# lianaR = .rho_scores(c(0.1, 0.3, 0.5, 0.1, 0.3))

In [ ]:
import numpy as np
import pandas as pd

from numpy.random import default_rng

from scipy.stats import beta


def beta_scores(rmat):
    rmat = np.sort(rmat, axis=0)
    n = rmat.shape[0]
    dist_a = np.repeat([np.repeat([np.arange(n)], rmat.shape[1], axis=0)], rmat.shape[2], axis=0).T + 1
    dist_b = n - dist_a + 1
    p = beta.cdf(np.sort(rmat, axis=0), dist_a, dist_b)
    return p


def corr_beta_pvals(p, k):
    p = np.clip(p * k, a_min=0, a_max=1)
    return p

In [ ]:
for method in _methods:
    if method.specificity is not None:
        # relevant_cols = [x for x in keys + [method.specificity, method.magnitude] if x is not None]
        lr = lr_res[method.method_name][keys + [method.specificity]]
        # discard any duplicates
        # this occurs when e.g. multiple subunits with expr. of 0 exist
        lr = lr.drop_duplicates(keep='first')
        lr_ranks[method.method_name] = lr
        # order = array.argsort()
        # ranks = order.argsort()
        # lr_ranks[method.method_name] = ranks


In [ ]:
joined = reduce(lambda left, right: pd.merge(left, right, how='outer', on=keys), lr_ranks.values())
specs = joined[['pvals', 'scaled_weight', 'lr_logfc', 'spec_weight']].values.T

In [ ]:
# Rank thing
rmat1 = ((specs.shape[1] - np.argsort(np.argsort(specs))) / (specs.shape[1])).T

In [ ]:
rmat

In [ ]:
# Desc:True/False; Method_name:Score, specificity, magnitude, steady (specificity + sca)

In [ ]:
lr_ranks={}

In [ ]:
for method in _methods:
    if method.magnitude is not None:
        # relevant_cols = [x for x in keys + [method.specificity, method.magnitude] if x is not None]
        lr = lr_res[method.method_name][keys + [method.magnitude]]
        # discard any duplicates
        # this occurs when e.g. multiple subunits with expr. of 0 exist
        lr = lr.drop_duplicates(keep='first')
        lr_ranks[method.method_name] = lr
        # order = array.argsort()
        # ranks = order.argsort()
        # lr_ranks[method.method_name] = ranks

In [ ]:
joined = reduce(lambda left, right: pd.merge(left, right, how='outer', on=keys), lr_ranks.values())

In [ ]:
joined # issue with non-unique: should iterateve over unique scores, not methods

In [ ]:
mags = joined[['lr_means_x', 'expr_prod_x', 'expr_prod_y','lr_means_y']].values.T
# Rank thing
rmat2 = ((mags.shape[1] - np.argsort(np.argsort(mags))) / (mags.shape[1])).T

In [ ]:
rmat = np.sort(rmat, axis=0)
n = rmat.shape[0]

In [ ]:
np.repeat(np.empty(1), 100)

In [ ]:
?np.repeat

In [ ]:
any(lr_res['CellPhoneDB'][keys].duplicated())

In [ ]:
any(joined.duplicated())

In [ ]:
lr_ranks['CellPhoneDB'][lr_ranks['CellPhoneDB'][keys].duplicated()]

In [ ]:
any(joined.duplicated())

In [ ]:
joined.drop_duplicates(keep='first', inplace=False)

In [ ]:
joined

In [ ]:
lr_res['CellPhoneDB']['x'] = 1

In [ ]:
all(joined.merge(lr_res['CellPhoneDB'], how='left')['x']==1)

In [ ]:
any(lr_res['CellPhoneDB'].duplicated())

In [ ]:
joined[joined[keys].duplicated()]

In [ ]:
any(np.isnan(joined['scaled_weight']))

In [ ]:
lr_ranks.values

In [ ]:
lr_ranks['CellPhoneDB']

In [ ]:
lr_ranks = {}

In [ ]:
np.zeros([5, 1000, 1000])

In [ ]:
for method in _methods:
    if method.specificity is not None:
        lr_ranks['specs'][method.method_name] = lr_res[method.method_name][method.specificity].values
    if method.magnitude is not None:
        lr_ranks['mags'][method.method_name] = lr_res[method.method_name][method.specificity].values

In [ ]:
lr_ranks

In [ ]:
l

## Each Method

CellPhoneDB

In [ ]:
from liana import cellphonedb

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

In [ ]:
lr_res = adata.uns['liana_res']

In [ ]:
lr_res[lr_res['receptor_complex']=='CD8A_CD8B']

NATMI

In [ ]:
from liana import natmi

In [ ]:
adata = natmi(adata, groupby='label')

In [ ]:
adata.uns['liana_res']

SCA

In [ ]:
from liana import singlecellsignalr as sca

In [ ]:
adata = sca(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_means', ascending=True)

Connectome

In [ ]:
from liana import connectome

In [ ]:
adata = connectome(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('scaled_weight', ascending=False)

logFC

In [ ]:
from liana import logfc

In [ ]:
adata = logfc(adata, groupby='label')

In [ ]:
adata.uns['liana_res'].sort_values('lr_logfc', ascending=False)

In [ ]:
np.exp(1)

try w/ real data

In [ ]:
adata = sc.datasets.pbmc3k_processed()
labels = adata.obs.louvain
cells = adata.obs_names

In [ ]:
adata = sc.datasets.pbmc3k()

In [ ]:
adata = adata[[x in cells for x in adata.obs_names]]

In [ ]:
adata.obs['label'] = labels

In [ ]:
sc.pp.log1p(adata)

In [ ]:
%%time
adata = cellphonedb(adata, groupby='label', n_perms=1000)

## LIANA PIPE LINE BY LINE

In [ ]:
# I get this from the score object
_complex_cols = ['ligand_means', 'receptor_means']
# change to full list and move to _var
_add_cols = ['ligand', 'receptor',
             'ligand_means_sums', 'receptor_means_sums',
             'ligand_zscores', 'receptor_zscores',
             'ligand_logfoldchanges', 'receptor_logfoldchanges',
             'ligand_logfc', 'receptor_logfc'
             'mat_mean',
             ]

_key_cols = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [ ]:
from liana.utils.pre import check_adata, check_if_covered, format_vars, filter_resource
from liana.resource import select_resource
from liana.resource.select_resource import explode_complexes
from liana.steady.liana_pipe import _get_lr
from liana.utils.reassemble_complexes import reassemble_complexes
from scipy.sparse import csr_matrix
from liana.steady._permutations import get_means_perms

I need to double check if every step makes sense, as I build unit tests!!!

For example, are there any duplicated rows (source, target, ligand_complex, receptor_complex)

Double check again if correct subunit is kept, etc

In [ ]:
adata = check_adata(adata, True)

In [ ]:
# Define idents col name
adata.obs.label = adata.obs[groupby]

Load resource

In [ ]:
resource = select_resource(resource_name='consensus')

In [ ]:
# Decomplexify
resource = explode_complexes(resource)

In [ ]:
# Filter Resource
resource = filter_resource(resource, adata.var_names)

Create entities

In [ ]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [ ]:
check_if_covered(entities, adata.var_keys)

Get global mean for SCA before filtering

In [ ]:
if 'mat_mean' in _add_cols: # SHOULD BE METHOD NAME!
    adata.uns['mat_mean'] = np.mean(adata.X)

In [ ]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

Get Stats

In [ ]:
lr_res = _get_lr(adata, resource, key_cols + _complex_cols + _add_cols, de_method)

In [ ]:
lr_res

Recomplexify

In [ ]:
# If I want to implement anything else but the min as complex policy, I would need to change this:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [ ]:
lr_res = reassemble_complexes(lr_res, _key_cols, _complex_cols)

In [ ]:
lr_res

### SCORES

In [ ]:
from liana.steady.Method import Method, MethodMeta
from liana.steady.scores.cellphonedb import _cpdb_score

CELLPHONEDB

In [ ]:
_cellphonedb = MethodMeta(method_name = "cellphonedb",
                          complex_cols = ['ligand_means', 'receptor_means'], 
                          add_cols = ['ligand', 'receptor'],
                          fun = _cpdb_score,
                          magnitude = 'lr_means',
                          specificity = 'pvals',
                          permute = True,
                          reference = 'Efremova et al., 2020')

In [ ]:
cellphonedb = Method(_SCORE=_cellphonedb)

In [ ]:
adata = cellphonedb(adata, groupby, seed=666)

In [ ]:
adata.uns['liana_res']

logFC re-implement

Magnitude as None

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

Both Specificity and Magnitude

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)